In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def get_avg_ratio(numbers):
    average = sum(numbers) / len(numbers)
    scaled = [x / average for x in numbers]
    scaled.sort(reverse=True)
    return scaled

In [3]:
lengths = [63, 73, 72, 60, 67, 66, 71, 61, 72, 70]

longest, *middle, shortest = get_avg_ratio(lengths)

print(f'Longest: {longest:>4.0%}')
print(f'Shortest: {shortest:>4.0%}')

### Item 20 Prefer Raising Exceptions to Returning *None*

In [4]:
def careful_divide(a,b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None
    

In [5]:
success, result = careful_divide(0,5)

if not success:
    print('invalid inputs')
else:
    print('Result is %.1f' % result)

Result is 0.0


In [6]:
def careful_division(a: float, b: float) -> float:
    """Divides a by b.
    Raises:
        ValueError: When the inputs cannot be divided."""
    
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

In [7]:
careful_division(0,5)

0.0

### Item 21 Know How Closures Interact with Vairable Scope

In [8]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key = helper)

In [9]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [3,2,7,5]
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


In [10]:
def sort_priority3(values, group):
    found = False
    def helper(x):
        nonlocal found    # Added
        if x in group:
            found = True
            return(0,x)
        return(1,x)
    values.sort(key=helper)
    return found


In [11]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [3,2,7,5]
found = sort_priority3(numbers, group)
print(f'Found: {found}')
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


In [12]:
# When your usage of nonlocal starts getting complicated, it’s better to
# wrap your state in a helper class.

class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)


In [13]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [3,2,8,6]

sorter = Sorter(group)
numbers.sort(key=sorter)
numbers
assert sorter.found is True

[2, 3, 6, 8, 1, 4, 5, 7]

### Item 22 Reduce Visual Noise with Variable Positional Arguments

In [14]:
def log1(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')


In [15]:
log1('My numbers are',[1,2,3])
log1('Hi there', [])

My numbers are: 1, 2, 3
Hi there


The bersome and noisy point in *log1* func is to pass an empty list when have no values to log. 
can do this in Python by prefixing the last positional
parameter name with \*. The first parameter for the log message is required, whereas any number of subsequent positional arguments are optional.

In [16]:
def log2(message, *values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

In [17]:
log2('My numbers are',1,2,3,5)
log2('Hi there')

My numbers are: 1, 2, 3, 5
Hi there


If I already have a sequence (like a list) and want to call a variadic
function like log, I can do this by using the \* operator.

In [18]:
favorites = [7,33,99]
log2('Favoriate colors', *favorites)

Favoriate colors: 7, 33, 99


There are two problems with accepting a variable number of positional arguments.

- The first issue is that these optional positional arguments are always
turned into a tuple before they are passed to a function. This means
that if the caller of a function uses the \* operator on a generator, it
will be iterated until it’s exhausted

In [19]:
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)


In [20]:
it = my_generator()
my_func(*it)

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


Functions that accept \*args are best for situations where you know
the number of inputs in the argument list will be reasonably small.
\*args is ideal for function calls that pass many literals or variable
names together. It’s primarily for the convenience of the programmer
and the readability of the code.

- The second issue with *args is that you can’t add new positional
arguments to a function in the future without migrating every caller.

In [21]:
def log3(sequence, message, *values):
    if not values:
        print(f'{sequence} - {message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{sequence} - {message}: {values_str}')

In [22]:
log3(1, 'Favorites', 7, 33) # New with *args OK
log3(1, 'Hi there') # New message only OK
log3('Favorite numbers', 7, 33) # Old usage breaks

1 - Favorites: 7, 33
1 - Hi there
Favorite numbers - 7: 33


Bugs like this are hard to track down because the code still runs without raising exceptions. To avoid this possibility entirely, you should use **keyword-only arguments** when you want to extend functions that accept \*args (see **Item 25**: “Enforce Clarity with Keyword-Only and Positional-Only Arguments”). To be even more defensive, you could also consider using type annotations (see **Item 90**: “Consider Static Analysis via typing to Obviate Bugs”).

**Things to Remember**

- Functions can accept a variable number of positional arguments by using \*args in the def statement.

- You can use the items from a sequence as the positional arguments for a function with the \* operator.

- Using the \* operator with a generator may cause a program to run out of memory and crash.

- Adding new positional parameters to functions that accept \*args can introduce hard-to-detect bugs.

### Item 23 Provide Optional Behavior with Keyword Arguments

In [23]:
def quotient(number, divisor):
    return number // divisor

assert quotient(20, 7) == 2

def remainder(number, divisor):
    return number % divisor

assert remainder(20, 7) == 6

The \** operator instructs Python to pass the values from a dictionary as the corresponding keyword arguments of the function

In [24]:
my_kwargs = {
    'number': 20,
    'divisor': 7,
}
assert remainder(**my_kwargs) == 6

In [25]:
# can use the ** operator multiple times if you know that the dic don't contain overlapping keys.
my_kwargs = {
    'number': 20
}
other_kwargs = {
    'divisor': 7
}

assert remainder(**my_kwargs, **other_kwargs) == 6

In [26]:
# use the **kwargs catch-all parameter to collect those arguments into a dict that you can then process.
def print_parameters(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')
print_parameters(alpha=1.5, beta=9, gamma=4)

alpha = 1.5
beta = 9
gamma = 4


### Item 24 Use *None* and Docstrings to Specify Dynamic Default Arguments

In [27]:
from time import sleep
from datetime import datetime

def log(message, when = datetime.now()):
    print(f'{when}: {message}')

In [28]:
log('Hi there!')
sleep(3)
log('Hello again!')

2021-03-05 09:24:31.309824: Hi there!
2021-03-05 09:24:31.309824: Hello again!


When the function is defined. The *default argument value* is evaluated only once per module load, which usually happens when a program starts up. 

After the module containing this code is loaded, the *datetime.now()* default argument will never be evaluated again.

In [29]:
def log2(message, when = None):
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

In [30]:
log2('Hi there!')
sleep(3)
log2('Hello again!')

2021-03-05 09:30:05.422487: Hi there!
2021-03-05 09:30:08.422823: Hello again!


Using *None* for default argument values is especially important when the arguments are mutable.


In [36]:
import json

def decode(data, default = {}):
    try:
        return json.loads(data)
    except ValueError:
        return default

The problem here is that the dictionary specified for *default* will be **shared by all calls to decode** because

default argument values are evaluated only once (at module load time).


In [39]:
foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 3
print('Foo: ', foo)
print('Bar:', bar)
assert foo is bar 

Foo:  {'stuff': 5, 'meep': 3}
Bar: {'stuff': 5, 'meep': 3}


The fix is to set the keyword argument default value to *None* and then document the behavior in the function's docstring.

In [40]:
def decode2(data, default = None):
    """Load JSON data from a string
    
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary."""
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

In [41]:
foo = decode2('bad data')
foo['stuff'] = 5
bar = decode2('also bad')
bar['meep'] = 3
print('Foo: ', foo)
print('Bar:', bar)
assert foo is not bar 

Foo:  {'stuff': 5}
Bar: {'meep': 3}


This approach also works with type annotations. 

Here, the *when* argument is marked as having an *Optional* vlaue that is datetime. 

Thus, the only two valid choices for *when* are *None* or a *datetime* object。

In [42]:
from typing import Optional

def log_typed(message: str, 
             when: Optional[datetime] = None) -> None:
    """Log a message with a timestamp.
    
    Args:
        message: Message to print.
        when: datetime of when the message Occurred.
            Defaults to the present time."""
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

In [43]:
log_typed('Hi there!')
sleep(3)
log_typed('Hello again!')

2021-03-05 09:56:29.673899: Hi there!
2021-03-05 09:56:32.674183: Hello again!


### Item 25 Enforce Clarity with Keyword-Only and Positional-Only Arguments

In [44]:
def safe_division(number, divisor,
                 ignore_overflow,
                 ignore_zero_division):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0 
        else: 
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [49]:
result = safe_division(1.0, 10**500, True, False)
result
result = safe_division(1.0, 0, True, True)
result

0

inf

It is better to require that callers are clear about their intentions by defining functions with ***keyword-only*** arguments. 

The **\*** symbol in the argument list indicates ***the end of positional arguments and the beginning of keyword-only arguments***. 

In [52]:
def safe_division_1(number, divisor, *,   # CHANGED
                 ignore_overflow = False,
                 ignore_zero_division = False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0 
        else: 
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [69]:
result = safe_division_1(1.0, 10**500, ignore_overflow = True, )
result
assert safe_division_1(1, 10**500) == 0

0

**Python 3.8** introduces a solution to this problem, called positional-only arguments. 

These arguments can be supplied only by position and never by keyword (the opposite of the keyword-only arguments demonstrated above).

The **\/** symbol in the argument list indicates where positional-only arguments end.

In [65]:
def safe_division_2(number, divisor, /, *,   # CHANGED
                 ignore_overflow = False,
                 ignore_zero_division = False):
    try:
        return number / divisor
    except OverflowError:
        if ignore_overflow:
            return 0 
        else: 
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [66]:
assert safe_division_2(2, 5) == 0.4

One notable consequence of keywork- and positional-only arguments is that any parameter name **between \/ and \* symbols in the argument list** may be passed either by position or by keyword (which is the default for all function arguments in Python). 

In [71]:
def safe_division_3(numerator, denominator, /, 
                    ndigits = 10, *,   # CHANGED
                 ignore_overflow = False,
                 ignore_zero_division = False):
    try:
        fraction = numerator / denominator
        return round(fraction, ndigits)
    except OverflowError:
        if ignore_overflow:
            return 0 
        else: 
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [75]:
result = safe_division_3(22,7)
result
result = safe_division_3(22,7,5)
result
result = safe_division_3(22,7,ndigits=2)
result

3.1428571429

3.14286

3.14

### Item 26 Define Function Decorators with *functools.wraps*

Python has special syntax for decorators that can be applied to functions. 

A decorator has the ability to run additional code before and after each call to a function it wraps. 

This means decorators can access and modify input arguments, return values, and raised exceptions.